In [1]:
### Program : Decisontree + Randfomforest + Cross Validation + gridsearch 
import os 
import numpy as np 
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
#import xgboost

In [16]:
%pwd

'D:\\Data Science - Course\\Random Forest'

In [17]:
os.chdir("D:\Data Science - Course\Random Forest")

In [18]:
chrn = pd.read_csv("Churn.csv")

In [19]:
chrn.head()

,Account Length,VMail Message,Day Mins,Eve Mins,Night Mins,Intl Mins,CustServ Calls,Churn,Intl Plan,VMail Plan,...,Day Charge,Eve Calls,Eve Charge,Night Calls,Night Charge,Intl Calls,Intl Charge,State,Area Code,Phone
0,128,25,265.1,197.4,244.7,10.0,1,0,0,1,...,45.07,99,16.78,91,11.01,3,2.70,KS,415,382-4657
1,107,26,161.6,195.5,254.4,13.7,1,0,0,1,...,27.47,103,16.62,103,11.45,3,3.70,OH,415,371-7191
2,137,0,243.4,121.2,162.6,12.2,0,0,0,0,...,41.38,110,10.30,104,7.32,5,3.29,NJ,415,358-1921
3,84,0,299.4,61.9,196.9,6.6,2,0,1,0,...,50.90,88,5.26,89,8.86,7,1.78,OH,408,375-9999
4,75,0,166.7,148.3,186.9,10.1,3,0,1,0,...,28.34,122,12.61,121,8.41,3,2.73,OK,415,330-6626


In [20]:
## drop the columns that are not required 
chrn = chrn.drop( [ "State", "Area Code", "Phone"], axis = 1)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [22]:
target = chrn["Churn"]
chrn = chrn.drop(["Churn"], axis=1)

In [23]:
## train and test split
x_train, x_test, y_train, y_test = train_test_split( chrn, target, test_size=0.2, random_state=42)

In [24]:
# Decision tree on churn data 
dt = DecisionTreeClassifier(max_depth = 8, min_samples_leaf=20,  random_state=1234)

In [25]:
## train the model 
dt.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1234, splitter='best')

In [26]:
## MOdel visualization 
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [27]:
## generate predictions
pred = dt.predict(x_test)

In [28]:
confusion_matrix( y_test, pred)

array([[551,  15],
       [ 31,  70]], dtype=int64)

In [29]:
## random Forest model 
rf = RandomForestClassifier()

In [30]:
rf = RandomForestClassifier(n_estimators=50, max_features= 5, class_weight='balanced',  max_depth=7,random_state = 123 )

In [31]:
rf.fit( x_train, y_train )

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=7, max_features=5,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=50, n_jobs=None, oob_score=False,
                       random_state=123, verbose=0, warm_start=False)

In [32]:
y_pred = rf.predict( x_test)

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix( y_test, y_pred)

array([[552,  14],
       [ 20,  81]], dtype=int64)

In [35]:
81/(81+14)

0.8526315789473684

In [36]:
81/(81+20)

0.801980198019802

In [37]:
2*0.87*0.8/(0.87+0.8)

0.8335329341317367

In [38]:
## variable importance of Randomforest model 
rf.feature_importances_

array([0.01828322, 0.02837568, 0.19185729, 0.05353186, 0.02559881,
       0.02609101, 0.20413722, 0.14088647, 0.01698128, 0.01628729,
       0.11654689, 0.01423247, 0.04435997, 0.01709173, 0.02130362,
       0.03339074, 0.03104444])

In [39]:
## Sorting variables on the importance 
import pandas as pd
feature_importances = pd.DataFrame(rf.feature_importances_, index = x_train.columns,columns=['importance']).sort_values('importance',  ascending=False)                                                          

In [40]:
feature_importances

,importance
CustServ Calls,0.204137
Day Mins,0.191857
Intl Plan,0.140886
Day Charge,0.116547
Eve Mins,0.053532
Eve Charge,0.044360
Intl Calls,0.033391
Intl Charge,0.031044
VMail Message,0.028376
Intl Mins,0.026091


In [41]:
import matplotlib.pyplot as plt
plt.plot(feature_importances)
plt.show()

<Figure size 640x480 with 1 Axes>

In [42]:
from sklearn.ensemble import GradientBoostingClassifier

In [43]:
## grid parameters for grid search
param_grid = {
    'max_depth': [6,7,8],
    'n_estimators': [10, 20, 30],
    'max_features' : [4,5,6]
}

In [46]:
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [47]:
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:    9.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [48]:
grid_search.best_params_

{'max_depth': 8, 'max_features': 6, 'n_estimators': 20}

In [50]:
grid_search.cv_results_

{'mean_fit_time': array([0.05779715, 0.10577183, 0.16394148, 0.06453156, 0.1195857 ,
        0.1819932 , 0.07081599, 0.14569736, 0.26011896, 0.0597549 ,
        0.11024127, 0.16884642, 0.06843128, 0.13240466, 0.20489116,
        0.07138376, 0.1494689 , 0.25824127, 0.06094494, 0.12312961,
        0.1743536 , 0.0734653 , 0.14259543, 0.21458697, 0.08800282,
        0.16517181, 0.24925718]),
 'std_fit_time': array([0.00510643, 0.00400183, 0.00502259, 0.00271146, 0.00385944,
        0.00633494, 0.00477502, 0.02152544, 0.03584901, 0.00431699,
        0.00273358, 0.011422  , 0.00364807, 0.01055908, 0.01035778,
        0.00537967, 0.00634207, 0.01970131, 0.00561972, 0.00430797,
        0.00619062, 0.00325823, 0.00633209, 0.00703457, 0.01123917,
        0.0050586 , 0.01879735]),
 'mean_score_time': array([0.00768695, 0.0052968 , 0.00896921, 0.00441351, 0.00546255,
        0.01017671, 0.00482011, 0.00603809, 0.01320724, 0.00522175,
        0.00981479, 0.01348948, 0.00079808, 0.00519147, 0.009397

In [52]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy = accuracy_score(test_labels,predictions)
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy


In [53]:

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, x_test, y_test)

Model Performance
Accuracy = 0.94%.


In [54]:
confusion_matrix(y_test,y_pred)

array([[552,  14],
       [ 20,  81]], dtype=int64)

In [ ]:
76/(76+5)

In [ ]:
76/(76+25)

In [ ]:
2*0.94*.75/(0.94+0.75)